In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [2]:
import OpenAI from "openai";

const openai = new OpenAI({
    apiKey: env["ALIBABA_API_KEY"],
});

In [3]:
function getCurrentWeather({ location, unit="fahrenheit"}){
   const  weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return JSON.stringify(weather_info);
}

In [4]:
const tools = [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: {
          type: "object",
          properties: {
            location: {
              type: "string",
              description: "The city and state, e.g. San Francisco, CA",
            },
            unit: { type: "string", enum: ["celsius", "fahrenheit"] },
          },
          required: ["location"],
        },
      },
    }
]

In [5]:
const messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样"
    }
]

const result = await openai.chat.completions.create({
    model: 'qwen-turbo',
    messages,
    tools
});
console.log(result.choices[0]);

Error: Connection error.

In [6]:
// 基于 langchain 使用 tools

import { z } from "zod";

const stringSchema = z.string();
stringSchema.parse("Hello, Zod!");


"Hello, Zod!"

In [7]:
// 基础类型
const stringSchema = z.string();
const numberSchema = z.number();
const booleanSchema = z.boolean();

// 数组
const stringArraySchema = z.array(z.string());
stringArraySchema.parse(["apple", "banana", "cherry"]); 

// 对象
const personSchema = z.object({
  name: z.string(),
  age: z.number(),
  // 可选类型
  isStudent: z.boolean().optional(),
  // 默认值
  home: z.string().default("no home")
});

// 联合类型
const mixedTypeSchema = z.union([z.string(), z.number()]);
mixedTypeSchema.parse("hello"); 
mixedTypeSchema.parse(42); 

42

In [8]:
const getCurrentWeatherSchema = z.object({
  location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  unit: z.enum(["celsius", "fahrenheit"]).describe("The unit of temperature"),
});

In [9]:
import { zodToJsonSchema } from "zod-to-json-schema";

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema);

In [15]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { convertToOpenAIFunction } from "@langchain/core/utils/function_calling";

const model = new ChatAlibabaTongyi({
    model: "qwen-turbo",
    temperature: 0,
})

const tools = [
    {
        type: "function",
        function: {
            name: "getCurrentWeather",
            description: "Get the current weather in a given location",
            parameters: zodToJsonSchema(getCurrentWeatherSchema),
        }
    }
];

const modelWithTools = model.bind({
  functions: tools.map((tool) => convertToOpenAIFunction(tool)),
});

await modelWithTools.invoke("北京的天气怎么样");

TypeError: Cannot read properties of undefined (reading '_def')

In [13]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const prompt = ChatPromptTemplate.fromMessages([
    ["system", "You are a helpful assistant"],
    ["human", "{input}"]
])

const chain = prompt.pipe(modelWithTools)

await chain.invoke({
    input: "北京的天气怎么样"
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "对不起，我无法获取实时的天气信息。你可以通过查看各大天气预报网站、使用专业的天气预报应用，或者询问本地的朋友来获得北京当前的天气情况。通常我会建议你关注气温、湿度、风向风力以及空气质量等信息，以做好适"... 5 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {},
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "对不起，我无法获取实时的天气信息。你可以通过查看各大天气预报网站、使用专业的天气预报应用，或者询问本地的朋友来获得北京当前的天气情况。通常我会建议你关注气温、湿度、风向风力以及空气质量等信息，以做好适"... 5 more characters,
  name: undefined,
  additional_kwargs: {},
  response_metadata: {
    tokenUsage: { promptTokens: 22, completionTokens: 59, totalTokens: 81 }
  },
  tool_calls: [],
  invalid_tool_calls: []
}